In [ ]:
# Setup OpenAI Agent
import openai

openai.api_key = "sk-your-key"

In [6]:
from llama_index.agent import OpenAIAgent

In [7]:
from typing import Optional, List
from llama_index.tools.tool_spec.base import BaseToolSpec
import ast


class PythonFileToolSpec(BaseToolSpec):
    spec_functions = ["function_definitions", "get_function", "get_functions"]

    def __init__(self, file_name: str) -> None:
        f = open(file_name).read()
        self.tree = ast.parse(f)

    def function_definitions(self, external: Optional[bool] = True) -> str:
        """
        Use this function to get the name and arguments of all function defintions in the python file
        """
        functions = ""
        for node in ast.walk(self.tree):
            if isinstance(node, ast.FunctionDef):
                if external and node.name.startswith("_"):
                    continue
                functions += f"""
name: {node.name}
arguments: {ast.dump(node.args)}
                    """
        return functions

    def get_function(self, name: str) -> str:
        for node in ast.walk(self.tree):
            if isinstance(node, ast.FunctionDef):
                if node.name == name:
                    return f"""
name: {node.name}
arguments: {ast.dump(node.args)}
docstring: {ast.get_docstring(node)}
                        """

    def get_functions(self, names: List[str]) -> str:
        functions = ""
        for name in names:
            functions += self.get_function(name) + "\n"
        return functions


pyfile = PythonFileToolSpec("./data/numpy_linalg.py")

In [ ]:
# Create the Agent with access to our tools
agent = OpenAIAgent.from_tools(
    pyfile.to_tool_list(),
    system_prompt="""
    You are a specialized agent will help create descriptions of tools that are informative and instruct a user how to use the tools.
    A tool is simply a function that will later be made callable by large language models like yourself.
    Generally tool prompts will describe the function of the tool, and provide information on the arguments and return values.
    
    Here is an example of a function definition and a tool prompt:
    def generate_image_variation(self, url: str, n: Optional[int] = 1, size: Optional[str] = '256x256') -> str:
        ```Accepts the url of an image and uses OpenAIs api to generate a variation of the image.
        This tool can take smaller images and create higher resolution variations, or vice versa.

        When passing a url from "generate_images" ALWAYS pass the url exactly as it was returned from the function, including ALL query parameters
        args:
            url (str): The url of the image to create a variation of
            n (Optional[int]): The number of images to generate. Defaults to 1.
            size (Optional[str]): The size of the image(s) to generate. Defaults to 256x256. Other accepted values are 1024x1024 and 512x512
        ```
    
    The first argument to all tools is self, as they are defined in a class

    If you load a list of function names and arguments, you should help the user by creating tool prompts like the above.
    
    In general, the entire prompt should not be more than 5-10 lines, and should be short to save on tokens
    """,
    verbose=True,
)

In [8]:
print(
    agent.chat(
        """
Load the eig, transpose and solve functions from the python file,
and then write a function defintion using ONLY! builtin python types (List, float, Tuple)
with a short 5-10 line doc string tool prompts for the functions that only has a small description and arguments
"""
    )
)

=== Calling Function ===
Calling function: get_function with args: {
  "name": "eig"
}
Got output: 
name: eig
arguments: arguments(posonlyargs=[], args=[arg(arg='a')], kwonlyargs=[], kw_defaults=[], defaults=[])
docstring: Compute the eigenvalues and right eigenvectors of a square array.

Parameters
----------
a : (..., M, M) array
    Matrices for which the eigenvalues and right eigenvectors will
    be computed

Returns
-------
A namedtuple with the following attributes:

eigenvalues : (..., M) array
    The eigenvalues, each repeated according to its multiplicity.
    The eigenvalues are not necessarily ordered. The resulting
    array will be of complex type, unless the imaginary part is
    zero in which case it will be cast to a real type. When `a`
    is real the resulting eigenvalues will be real (0 imaginary
    part) or occur in conjugate pairs

eigenvectors : (..., M, M) array
    The normalized (unit "length") eigenvectors, such that the
    column ``eigenvectors[:,i]`` is 

In [ ]:
"""Numpy tool spec."""

from llama_index.tools.tool_spec.base import BaseToolSpec
from typing import Optional, List, Tuple, Union
import numpy as np

class NumpyToolSpec(BaseToolSpec):
    """Numpy Tool Spec"""

    spec_functions = ["compute_eigenvalues_and_eigenvectors", "transpose_matrix", "solve_linear_equation"]

        def compute_eigenvalues_and_eigenvectors(a: List[List[float]]) -> Tuple[List[complex], List[List[complex]]]:
        """
        Compute the eigenvalues and right eigenvectors of a square array.
    
        Parameters:
            a (List[List[float]]): Matrices for which the eigenvalues and right eigenvectors will be computed
    
        Returns:
            Tuple[List[complex], List[List[complex]]]: A tuple containing the eigenvalues and eigenvectors.
                The eigenvalues are a list of complex numbers, each repeated according to its multiplicity.
                The eigenvectors are a list of lists, where each inner list represents a normalized eigenvector.
                The column eigenvectors[i] is the eigenvector corresponding to the eigenvalue eigenvalues[i].
        """
        pass


    def transpose_matrix(self, a: List[List[float]]) -> List[List[float]]:
        """
        Transpose each matrix in a stack of matrices.

        Parameters:
            a (List[List[float]]): The matrix to transpose

        Returns:
            List[List[float]]: The transposed matrix
        """
        return np.transpose(a)


    def solve_linear_equation(self, a: List[List[float]], b: Union[List[float], List[List[float]]]) -> Union[List[float], List[List[float]]]:
        """
        Solve a linear matrix equation, or system of linear scalar equations.

        Parameters:
            a (List[List[float]]): Coefficient matrix.
            b (Union[List[float], List[List[float]]]): Ordinate or "dependent variable" values.

        Returns:
            Union[List[float], List[List[float]]]: Solution to the system a x = b.
                The returned shape is identical to b.
        """
        return np.linalg.solve(a, b)


In [ ]:
from llama_index.agent import OpenAIAgent

agent = OpenAIAgent.from_tools(NumpyToolSpec().to_tool_list(), verbose=True)

In [ ]:
print(
    agent.chat(
        """
Using the tools provided, solve the system of equations ``x0 + 2 * x1 = 1`` and ``3 * x0 + 5 * x1 = 2``, then transpose the coefficent matrix and compute the eigenvalues
"""
    )
)